# QLoRa 实战

## Step1 导入相关包

In [14]:
## 学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [15]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [16]:
ds = Dataset.load_from_disk("/root/tuning/lesson01/data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:1]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

## Step3 数据集预处理

In [17]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [18]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [19]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

## Step4 创建模型

In [20]:
import torch
##修改
# low_cpu_mem_usage=True: 这个参数设定为True意味着在模型加载时会尽可能地减少CPU内存的使用。
# torch_dtype=torch.half: 这个参数设置了模型中张量的数据类型为半精度浮点数，这可以减少内存占用和计算时间，但可能会牺牲一些精度。
# device_map="auto": 这个参数设置了模型应该在哪个设备上运行。“auto”意味着它将自动选择可用的设备，优先选择GPU，如果没有GPU则选择CPU。
# load_in_4bit=True: 这个参数设置为True意味着在模型加载时将使用4位量化，这可以进一步减少内存占用。
# bnb_4bit_compute_dtype=torch.half: 这个参数设置了在4位量化时的计算数据类型，这里设置为半精度浮点数。
# bnb_4bit_quant_type="nf4": 这个参数设置了4位量化的类型，"nf4"是一种特定的量化策略。
# bnb_4bit_use_double_quant=True: 这个参数设置为True意味着在4位量化时使用双重量化。
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",
                                              torch_dtype=torch.half,
                                              low_cpu_mem_usage=True, 
                                              device_map="auto", 
                                              load_in_4bit=True,
                                              bnb_4bit_quant_type="nf4", 
                                              bnb_4bit_use_double_quant=True)


In [10]:
model.dtype

torch.float16

In [13]:
for name, parameter in model.named_parameters():
    print(name,parameter.dtype)

transformer.word_embeddings.weight torch.float16
transformer.word_embeddings_layernorm.weight torch.float16
transformer.word_embeddings_layernorm.bias torch.float16
transformer.h.0.input_layernorm.weight torch.float16
transformer.h.0.input_layernorm.bias torch.float16
transformer.h.0.self_attention.query_key_value.weight torch.uint8
transformer.h.0.self_attention.query_key_value.bias torch.float16
transformer.h.0.self_attention.dense.weight torch.uint8
transformer.h.0.self_attention.dense.bias torch.float16
transformer.h.0.post_attention_layernorm.weight torch.float16
transformer.h.0.post_attention_layernorm.bias torch.float16
transformer.h.0.mlp.dense_h_to_4h.weight torch.uint8
transformer.h.0.mlp.dense_h_to_4h.bias torch.float16
transformer.h.0.mlp.dense_4h_to_h.weight torch.uint8
transformer.h.0.mlp.dense_4h_to_h.bias torch.float16
transformer.h.1.input_layernorm.weight torch.float16
transformer.h.1.input_layernorm.bias torch.float16
transformer.h.1.self_attention.query_key_value.we

## Lora

### PEFT Step1 配置文件

In [21]:
from peft import LoraConfig, TaskType, get_peft_model
## ,target_modules=["query_key_value"],r=8
config = LoraConfig(task_type=TaskType.CAUSAL_LM,r=8, target_modules=['query_key_value'])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['query_key_value'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [22]:
# 在深度神经网络 [deep neural network] 训练时，需要对每个参数或权重 [parameter/weight] 计算其对损失函数 
# [loss function] 的梯度 [gradient]，从而进行反向传播 [back propagation] 和优化[optimization]。
# 默认情况下不会计算输入数据 [input data] 的梯度，即使它们在计算中起到了关键的作用。但是，在某些应用场景中，
# 例如图像生成 [image generation]、注意力机制 [attention mechanism] 等，需要计算输入数据的梯度。此时，
# 可以通过启用计算输入梯度的功能，对输入数据进行求导并利用其梯度信息进行优化。
# 作用: 启用该功能这对于在保持模型权重固定的同时微调适配器权重非常有用。

model.enable_input_require_grads()

### PEFT Step2 创建模型

In [23]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(46145, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear4bit(
                in_features=2048, out_features=6144, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
               

In [29]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value', 'dense_4h_to_h'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

## Step5 配置训练参数

In [24]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/tuningdata/qlora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"
)

## Step6 创建训练器

In [25]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [26]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
20,2.804900
40,2.715800
60,2.591400
80,2.541700
100,2.436600
120,2.471500
140,2.487400
160,2.453200
180,2.436000
200,2.374700


TrainOutput(global_step=839, training_loss=2.3699914798122763, metrics={'train_runtime': 933.0614, 'train_samples_per_second': 28.785, 'train_steps_per_second': 0.899, 'total_flos': 1.1645495978754048e+16, 'train_loss': 2.3699914798122763, 'epoch': 1.0})

## Step8 模型推理

In [38]:
from peft import PeftModel
from transformers import pipeline

model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

p_model = PeftModel.from_pretrained(model=model, model_id="/root/autodl-tmp/tuningdata/qlora/checkpoint-500")

pipe = pipeline("text-generation", model=p_model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=500, do_sample=True, )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Refor

[{'generated_text': 'Human: 如何写好一个简历？\n\nAssistant: 好的，那么你应该考虑以下几点：\n\n1. 职位相关性\n\n有些职位可能会要求你具有相应的学历或工作经验，所以你需要在简历中附上这些信息，以确保你不会被误解为没有相关经验或学历。\n\n2. 个人信息部分\n\n在你的个人信息部分上，一定要附上你自述的职位，并提供你详细的职位描述。\n\n3. 背景与工作经历\n\n在这里你可以列出你过去的工作经历，包括工作项目、取得的奖励、你的发展方向、参加过的课程等。\n\n4. 优势项目\n\n除了上面提到的经历外，你还可以补充一些你擅长的项目，这样你就可以更容易让招聘人员了解到你的特质并做出判断。\n\n5. 能力证明部分\n\n这里你需要附上你过去工作中涉及到的关键工具和流程，以及通过这些工具和流程实现的实际结果。\n\n6. 本职工作领域\n\n你还应该附上你的主要工作领域，这样招聘人员就可以了解你的技能、经验和知识在哪些领域发挥着作用。\n\n7. 专长描述部分\n\n在这些方面，你可以描述一下你在这个专业领域拥有过哪些独特的技能，哪些领域你比其他人更有优势，以及有哪些是你自己擅长的。\n\n8. 职位描述部分\n\n在这里你可以附上你在当前工作领域取得的成就，描述下你在这项工作中能够为公司带来的价值，并证明你能够胜任这份工作。\n\n9. 未来的发展规划\n\n除了这个部分，你也可以补充一些未来的发展规划，这样招聘人员就可以了解你的目标和野心。\n\n10. 联系方式\n\n这里你可以附上你的联络方式，以便招聘人员能够及时与你联系，讨论相关事宜。\n\n11. 备注部分\n\n在简历的最后，你可以附上一个个人备注部分，你可以在这里说明如何能够更好的帮助面试者了解你，并阐述下你想找工作的原因。'}]